In [ ]:
#Compute limits
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jan  7 03:53:35 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   42C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install rouge_score
!pip install peft
!pip install trl
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=17b2815bf154a8bf0663c82f

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import statistics
import json
import os
import transformers
import torch
from datasets import load_dataset, Dataset, DatasetDict
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

# Load Gemma 2B Model

In [ ]:
# Download Gemma 2b-it base model and tokenizer from huggingface
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            quantization_config = bnb_config,
                                            device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Configure LoRA
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM"
)

# Datasets Processing

In [ ]:
# Load the train and validation datasets into Dataframes.
df_train = pd.read_csv('/content/train.csv')
df_val = pd.read_csv('/content/validation.csv')

In [ ]:
# Function to create a new column "Extract_Chunk" in the train and val dataframes.
# Extracts last sentence from all parts of the story (for last part, it is "") and copies to new column "Extract_Chunk"
# The last sentence in each part of the story becomes prompt to the next part. The last part of every story ends with "కథ సమాప్తం"

def Extract_Chunk(df):
  df['Extract_Chunk'] = df['Story'].apply(lambda x: '' if x.split('.')[-2] == '\nకథ సమాప్తం'
                                              else ''.join([l.strip() + '. ' for l in x.split('.')[-2:-1]]))

In [ ]:
# Function to create a new column "To_Tokenize" in the train and val dataframes.
# 'To_Tokenize' column is concatenation of the following:
#  for part 1 of story : Prompt to the story + Story Title + Actual Story
#  for remaining parts of the story : Last sentence of the previous part as prompt + Story

def To_Tokenize(df):
  for i in range(len(df)):
    if df.at[i, 'Part'] == 'ఒకటి':
      df.at[i,'To_Tokenize'] = 'Prompt: ' + df.at[i,'Prompt'] + '\n' + 'Title: ' + df.at[i, 'Title'] + '\n' + 'Story: ' + df.at[i, 'Story'] + '<eos>'
    else:
      df.at[i, 'To_Tokenize'] = 'Prompt: ' + df.at[i-1,'Extract_Chunk']+ '\n' + df.at[i, 'Story'] + '<eos>'


In [ ]:
# Pass the dataframes to the functions to create "Extract_Chunk" and "To_Tokenize" columns.
Extract_Chunk(df_train)
Extract_Chunk(df_val)
To_Tokenize(df_train)
To_Tokenize(df_val)

In [ ]:
# Verify - total number of tokens is less than 1024 in both train and validation dataframes.
df_val['Story_len'] = df_val['To_Tokenize'].apply(lambda x: len(tokenizer(x)['input_ids']))
df_val['Story_len'].describe()

,Story_len
count,33.000000
mean,666.848485
std,154.653266
min,378.000000
25%,554.000000
50%,681.000000
75%,764.000000
max,992.000000


In [ ]:
# Function to convert 'To_Tokenize' column into JSON-formatted string and write to a file.
def write_json_file(df, file):
  data=[]
  for index, row in df.iterrows():
    data += [{'to_tokenize': row.iloc[5]}]
  with open(file+'_file.json', 'w') as file:
    json.dump(data, file)

write_json_file(df_train, 'train')
write_json_file(df_val, 'val')

In [ ]:
# Create Tokenized train and validation datasets

from datasets import load_dataset
dataset = load_dataset('json', data_files = {'train':'train_file.json', 'validation':'val_file.json'})
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):

    inputs = [pro for pro in examples['to_tokenize']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = model_inputs['input_ids'].copy()
    model_inputs['labels'] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function,  batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

# Fine-tuning

In [ ]:
from transformers import Trainer, TrainingArguments

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Training arguments configuration
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",    # Evaluate model at the end of each epoch.
    learning_rate=2e-4,             # Learning rate for optimization.
    per_device_train_batch_size=2,  # Number of samples per batch during training.
    per_device_eval_batch_size=2,   # Number of samples per batch during evaluation.
    num_train_epochs=15,            # Total number of training epochs.
    weight_decay=0.01,              # Regularization to prevent overfitting.
    save_total_limit=1,             # Limit the number of checkpoints saved to 1. Delete older checkpoints.
    logging_dir='./logs',           # Directory for logging training metrics.
    logging_strategy='epoch',       # Logs metrics at the end of each epoch.
    save_strategy = 'epoch',        # Saves the model checkpoint at the end of each epoch.
    report_to="none",               # Disables reporting to external logging systems like 'WandB'.
    fp16=True,                      # Enable mixed precision training for faster computation and lower memory usage on GPUs.
    optim="adamw_torch"             # optimizer - AdamW to be used during training.
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],      # The tokenized dataset used for training,
    eval_dataset=tokenized_dataset['validation'],  # The tokenized dataset used for validation during training.
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,1.370300,1.258575
2,1.062300,1.020963
3,0.797200,0.759197
4,0.558100,0.556329
5,0.382100,0.358884
6,0.246900,0.211873
7,0.144800,0.120619
8,0.078600,0.066544
9,0.043400,0.038610
10,0.025300,0.027027


TrainOutput(global_step=3435, training_loss=0.3176760318497904, metrics={'train_runtime': 4007.3464, 'train_samples_per_second': 1.711, 'train_steps_per_second': 0.857, 'total_flos': 8.38842568409088e+16, 'train_loss': 0.3176760318497904, 'epoch': 15.0})

In [ ]:
# Save the fine-tuned model and tokenizer.
import shutil

OUT_DIR = "finetuned_model"
model.save_pretrained(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

# Compress the directory
shutil.make_archive(OUT_DIR, 'zip', OUT_DIR)

'/content/finetuned_model.zip'

In [ ]:
# Download the  finetuned model ZIP file.
from google.colab import files
files.download(f"{OUT_DIR}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>